# n2c2

## Checking the annotations file

In [1]:
n2c2_ann = [line.strip().split("\t") for line in open("Downloads/data_sample/164726.ann", "r")]

In [3]:
n2c2_result = []
for item in n2c2_ann:
    words = item[1].split()
    try:
    # check if the second last word is a number
        if words[-2].isdigit() and words[-1].isdigit():
            start, end = map(int, [words[-2], words[-1]])
            # check if the length of the string in the 3rd position is equal to the difference between the two numbers
            if end - start == len(item[2]):
                n2c2_result.append([item[0], words[0], f"{start}:{end}", item[2]])
        
    except:
        continue

print(n2c2_result[:10])

[['T1', 'Drug', '1341:1344', 'TPA'], ['T2', 'Route', '1376:1378', 'IV'], ['T3', 'Drug', '1379:1384', 'nitro'], ['T4', 'Drug', '1397:1404', 'heparin'], ['T5', 'Drug', '1406:1413', 'aspirin'], ['T6', 'Reason', '5420:5428', 'diuresis'], ['T7', 'Route', '5434:5436', 'IV'], ['T8', 'Drug', '5437:5442', 'lasix'], ['T9', 'Drug', '5447:5453', 'diuril'], ['T10', 'Reason', '5524:5548', 'hemodynamically unstable']]


## Merging word annotations and text

In [94]:
def load_text(file_path):
    with open(file_path, "r") as file:
        return file.read()

def load_annotations(file_path):
    annotations = []
    with open(file_path, "r") as file:
        for line in file:
            items = line.strip().split("\t")
            try:
                annotations.append([items[0], items[1], items[2].split()])
            except:
                continue
    return annotations

def annotated_text(text, annotations):
    positions = []
    for annotation in annotations:
        if annotation[0].startswith("T"):
            start, end = map(int, annotation[1].split()[1:])
            positions.append((start, end, annotation[1].split(" ")[0], annotation[2]))

    # sort the positions list by start index
    positions.sort(key=lambda x: x[0])

    annotated_text = ""
    start = 0
    for position in positions:
        annotated_text += text[start:position[0]]
        annotated_text += f"<{position[2]}> {str(position[3])} </{position[2]}> "
        start = position[1]
    annotated_text += text[start:]
    return annotated_text


In [5]:
text = load_text("Downloads/data_sample/164726.txt")
annotations = load_annotations("Downloads/data_sample/164726.ann")
txt = annotated_text(text, annotations)

In [10]:
def bart_to_ner(text):
    tokens = text.split()
    ner_tags = []
    entity_start = False
    current_entity = ""
    current_tag = "0"
    
    for token in tokens:
        if "<" in token and ">" in token:
            if "/" in token:
                entity_start = False
                ner_tags.append((current_entity, current_tag))
                current_entity = ""
                current_tag = "0"
            else:
                entity_start = True
                current_tag = token.split("<")[1].split(">")[0]
        elif entity_start:
            current_entity += token + " "
        else:
            ner_tags.append((token, 0))
    return ner_tags

ner_txt = bart_to_ner(txt)


## Turning annotated text into NER annotations

In [87]:
import re

nerred_txt = []

for i, j in enumerate(ner_txt):
    if j[1] == "ADE":
        words = re.findall(r'\b\w+\b', "B-"+j[0])
        nerred_txt.append((words[0], j[1]))
        for word in words[1:]:
            nerred_txt.append((word, "B-"+j[1]))
    else:
        words = re.findall(r'\b\w+\b', j[0])
        for word in words:
            nerred_txt.append((word, 0))


In [13]:
for i, j in enumerate(nerred_txt):
    if j[1]!=0:
        print(i, j)

937 ('metabolic', 'ADE')
938 ('acidosis', 'ADE')


## ALL the files

In [103]:
ner_texts_eng=[]
for nmbr in range(100035,198407):
    try:
        text = load_text(f"Downloads/training_20180910/{nmbr}.txt")
        annotations = load_annotations(f"Downloads/training_20180910/{nmbr}.ann")
        txt = annotated_text(text, annotations)
        ner_txt = bart_to_ner(txt)

        nerred_txt = []

        for i, j in enumerate(ner_txt):
            if j[1] == "ADE":
                words = re.findall(r'\b\w+\b', "B-"+j[0])
                nerred_txt.append((words[0], j[1]))
                for word in words[1:]:
                    nerred_txt.append((word, "B-"+j[1]))
            else:
                words = re.findall(r'\b\w+\b', j[0])
                for word in words:
                    nerred_txt.append((word, 0))
        ner_texts_eng.append(nerred_txt)
    except:
        continue

# Estonian stuff

In [37]:
import pandas as pd

In [16]:
tekst= pd.read_csv("Downloads/training_data_AD_and_AP_1st_and_2nd.csv")

In [92]:
import re

def NER_annotate(text):
    adr_pattern = r'<adr> (\w+) <adr>'
    annot_pattern= r'<(\w+)>'

    adr_matches = re.findall(adr_pattern, text)
    annot_matches=re.findall(annot_pattern, text)

    words = re.findall(r'\b\w+\b', text)

    ner_annotated_text = []
    for word in words:
        if word in adr_matches:
            ner_annotated_text.append((word, 'B-ADE'))
        elif word in annot_matches:
            continue
        else:
            ner_annotated_text.append((word, 0))

    return ner_annotated_text

In [90]:
tekstid= [NER_annotate(tekst.iloc[i]["paragraph_text"]) for i in range(len(tekst))][1:]